In [5]:
import keyboard
import sounddevice as sd
import queue
import json
import pyperclip
import tkinter as tk
import re
from vosk import Model, KaldiRecognizer


MODEL_PATH = "vosk-model-en-us-0.42-gigaspeech"  
HINDI_MODEL_PATH = "vosk-model-small-hi-0.22" 

english_model = Model(MODEL_PATH)
hindi_model = Model(HINDI_MODEL_PATH)

# Queue to hold audio data
audio_queue = queue.Queue()

def callback(indata, frames, time, status):
    if status:
        print(status)
    audio_queue.put(bytes(indata))

def clean_text(text):
    if len(text) == 1:
        return text 
    text = text.capitalize()  
    text = re.sub(r"\s+", " ", text).strip() 
    text = re.sub(r"(\w)([,.!?])", r"\1\2", text) 
    if text and text[-1] not in ".!?":
        text += "."  
    return text

def listen_and_transcribe(language="english"):
    model = english_model if language == "english" else hindi_model
    recognizer = KaldiRecognizer(model, 16000)
    recognizer.SetWords(True)
    
    with sd.RawInputStream(samplerate=16000, blocksize=8000, dtype='int16', channels=1, callback=callback):
        print("Listening... Speak now!")
        full_text = ""
        
        while True:
            data = audio_queue.get()
            if recognizer.AcceptWaveform(data):
                result = json.loads(recognizer.Result())
                text = result.get("text", "").strip()
                if text:
                    full_text = clean_text(text)
                    break  
            else:
                partial_result = json.loads(recognizer.PartialResult()).get("partial", "").strip()
                if partial_result:
                    full_text = clean_text(partial_result)  # Show live text

        if full_text:
            print("You said:", full_text)
            pyperclip.copy(full_text)  
            text_output.insert(tk.END, full_text + "\n") 
            with open("voice_history.txt", "a", encoding="utf-8") as f:
                f.write(full_text + "\n")  

def discard_text():
    text_output.delete("1.0", tk.END)  # Clear text
    tk_root.withdraw()  # Hide the UI

def show_ui():
    tk_root.deiconify()  # Show the UI
    with open("voice_history.txt", "r", encoding="utf-8") as f:
        history = f.read()
    text_output.delete("1.0", tk.END)
    text_output.insert(tk.END, history) 
    listen_and_transcribe(language=language_var.get())

# Create UI
tk_root = tk.Tk()
tk_root.title("Voice Clipboard")
tk_root.geometry("500x350")
tk_root.configure(bg="#1e1e1e")
tk_root.withdraw()  # Hide initially

text_output = tk.Text(tk_root, bg="#252526", fg="#ffffff", font=("Arial", 12), wrap=tk.WORD)
text_output.pack(expand=True, fill=tk.BOTH, padx=10, pady=10)

language_var = tk.StringVar(value="english")
language_frame = tk.Frame(tk_root, bg="#1e1e1e")
language_frame.pack(pady=5)

tk.Radiobutton(language_frame, text="English", variable=language_var, value="english", fg="white", bg="#1e1e1e").pack(side=tk.LEFT, padx=10)
tk.Radiobutton(language_frame, text="Hindi", variable=language_var, value="hindi", fg="white", bg="#1e1e1e").pack(side=tk.LEFT, padx=10)

discard_button = tk.Button(tk_root, text="Discard", command=discard_text, bg="#ff5555", fg="#ffffff")
discard_button.pack(pady=5)


keyboard.add_hotkey("alt+ctrl", show_ui)

print("Voice Clipboard running... Press Alt + Ctrl to start speaking.")
tk_root.mainloop()

ModuleNotFoundError: No module named 'keyboard'

In [ ]:
pip install keyboard


Note: you may need to restart the kernel to use updated packages.


: 

In [3]:
pip install --upgrade pip

  Using cached pip-25.0.1-py3-none-any.whl.metadata (3.7 kB)
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 1.8/1.8 MB 25.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.3.1
    Uninstalling pip-24.3.1:
      Successfully uninstalled pip-24.3.1
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [6]:
pip install pyttsx3


Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
